In [10]:
import tensorflow as tf
import os
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from imutils import paths
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from Augmentor import Pipeline

In [ ]:
def augment_images(data_path: str, augment_count: int) -> None:

    sample = Augmentor.Pipeline(data_path)
    Pipeline.set_seed(1230)
    sample.resize(probability=1.0, width=160, height=120, resample_filter="BICUBIC")
    sample.rotate(probability=0.5, max_left_rotation=10, max_right_rotation=10)
    sample.zoom(probability=0.5, min_factor=1.0, max_factor=1.2)
    sample.flip_left_right(probability=0.5)
    sample.gaussian_distortion(
        probability=1,
        grid_width=4,
        grid_height=4,
        magnitude=2,
        corner="bell",
        method="in",
        mex=0.5,
        mey=0.5,
        sdx=0.05,
        sdy=0.05,
    )
    sample.random_brightness(probability=1, min_factor=0.5, max_factor=1.5)
    sample.random_color(probability=1, min_factor=0.5, max_factor=2)

    sample.status()
    sample.sample(augment_count, multi_threaded=True)
    return

In [ ]:
# EXAMPLE AUGMENTATION FOR TRAINING DATASET
augment_images('/home/z6/Furkan-data/seatbelt/REFINED_RAW_DATASET/train', 300_000)

In [2]:
# 300_000 AUGMENTED DATA
TRAIN_PATH = '/home/z6/Furkan-data/seatbelt/aug-refined-original/train'
VAL_PATH = '/home/z6/Furkan-data/seatbelt/aug-refined-original/val'
TEST_PATH = '/home/z6/Furkan-data/seatbelt/aug-refined-original//test'

# determine the total number of image paths in training, validation,
# and testing directories
totalTrain = len(list(paths.list_images(TRAIN_PATH)))
totalVal = len(list(paths.list_images(VAL_PATH)))
totalTest = len(list(paths.list_images(TEST_PATH)))

batch_size = 256

train_datagen = ImageDataGenerator(rescale=1 / 255.0)


trainGen = train_datagen.flow_from_directory(
    TRAIN_PATH,
    class_mode="categorical",
    target_size=(160, 120),
    color_mode="rgb",
    shuffle=True,
    batch_size=batch_size)

val_datagen = ImageDataGenerator(rescale=1 / 255.0)

valGen = val_datagen.flow_from_directory(
    VAL_PATH,
    class_mode="categorical",
    target_size=(160, 120),
    color_mode="rgb",
    shuffle=True,
    batch_size=batch_size)

testAug = ImageDataGenerator(rescale=1 / 255.0)

testGen = testAug.flow_from_directory(
    TEST_PATH,
    class_mode="categorical",
    target_size=(160, 120),
    color_mode="rgb",
    shuffle=False,
    batch_size=batch_size)

Found 240000 images belonging to 2 classes.
Found 30000 images belonging to 2 classes.
Found 30000 images belonging to 2 classes.


In [19]:
class LossAndErrorPrintingCallback(tf.keras.callbacks.Callback):
    def on_train_batch_end(self, batch, logs=None):
        print("For batch {}, loss is {:7.4f}.".format(batch, logs["loss"]))

    def on_test_batch_end(self, batch, logs=None):
        print(" For batch {}, loss is {:7.4f}.".format(batch, logs["loss"]))


In [20]:
opt = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model = tf.keras.applications.InceptionV3(
    include_top=True, weights=None, input_tensor=None,
    input_shape=(160, 120, 3), pooling=None, classes=2)

model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['accuracy'])

In [21]:
epochs = 10
batch_size = 256
starter = model.fit_generator(
    trainGen,
    steps_per_epoch=totalTrain // batch_size,
    validation_data=valGen,
    validation_steps=totalVal // batch_size,
    validation_freq=1,
    epochs=epochs,
callbacks=[LossAndErrorPrintingCallback()])

Train for 937 steps, validate for 117 steps
Epoch 1/10
For batch 0, loss is  0.8002.
 69/937 [=>............................] - ETA: 9:02 - loss: 0.2066 - accuracy: 0.8956For batch 69, loss is  0.0157.


207/937 [=====>........................] - ETA: 6:50 - loss: 0.0792 - accuracy: 0.9615For batch 207, loss is  0.0009.


345/937 [==========>...................] - ETA: 5:16 - loss: 0.0497 - accuracy: 0.9762For batch 345, loss is  0.0004.


483/937 [==============>...............] - ETA: 3:57 - loss: 0.0371 - accuracy: 0.9825For batch 483, loss is  0.0019.


621/937 [==================>...........] - ETA: 2:42 - loss: 0.0293 - accuracy: 0.9863For batch 621, loss is  0.0134.


...

503/937 [===============>..............] - ETA: 3:34 - loss: 3.3785e-06 - accuracy: 1.0000For batch 503, loss is  0.0000.


637/937 [===================>..........] - ETA: 2:28 - loss: 3.3078e-06 - accuracy: 1.0000For batch 637, loss is  0.0000.


771/937 [=======================>......] - ETA: 1:21 - loss: 3.2966e-06 - accuracy: 1.0000For batch 771, loss is  0.0000.


905/937 [===========================>..] - ETA: 15s - loss: 3.2767e-06 - accuracy: 1.0000For batch 905, loss is  0.0000.


936/937 [============================>.] - ETA: 0s - loss: 3.2809e-06 - accuracy: 1.0000For batch 936, loss is  0.0000.
 For batch 0, loss is  0.0203.
 For batch 1, loss is  0.0494.
 For batch 2, loss is  0.0136.
 For batch 3, loss is  0.0211.
 For batch 4, loss is  0.0099.
 For batch 5, loss is  0.0163.
 For batch 6, loss is  0.0011.
 For batch 7, loss is  0.0476.
 For batch 8, loss is  0.0180.
 For batch 9, loss is  0.0243.
 For batch 10, loss is  0.0289.
 For batch 11, loss is  0.0226.
 For batch 12, loss is  0.0017.
 For batch 13, loss is  0.0106.
 For batch 14, loss is  0.0064.
 For batch 15, loss is  0.0147.
 For batch 16, loss is  0.0145.
 For batch 17, loss is  0.0401.
 For batch 18, loss is  0.0006.
 For batch 19, loss is  0.0166.
 For batch 20, loss is  0.0235.
 For batch 21, loss is  0.0489.
 For batch 22, loss is  0.0347.
 For batch 23, loss is  0.0252.
 For batch 24, loss is  0.0029.
 For batch 25, loss is  0.0225.
 For batch 26, loss is  0.0198.
 For batch 27, loss is  0.

In [22]:
testGen.reset()
predIdxs = model.predict_generator(testGen,
                                        steps=(totalTest // batch_size) + 1)
predIdxs = np.argmax(predIdxs, axis=1)

print(classification_report(testGen.classes, predIdxs, target_names=testGen.class_indices.keys()))
print(confusion_matrix(testGen.classes, predIdxs))


              precision    recall  f1-score   support

    negative       1.00      1.00      1.00     14969
    positive       1.00      1.00      1.00     15031

    accuracy                           1.00     30000
   macro avg       1.00      1.00      1.00     30000
weighted avg       1.00      1.00      1.00     30000

[[14952    17]
 [   18 15013]]


In [23]:
model = tf.keras.applications.DenseNet121(
    include_top=True, weights=None, input_tensor=None,
    input_shape=(160, 120, 3), pooling=None, classes=2)

model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['accuracy'])

In [24]:
starter = model.fit_generator(
    trainGen,
    steps_per_epoch=totalTrain // batch_size,
    validation_data=valGen,
    validation_steps=totalVal // batch_size,
    validation_freq=1,
    epochs=epochs,
callbacks=[LossAndErrorPrintingCallback()])

Train for 937 steps, validate for 117 steps
Epoch 1/10
For batch 0, loss is  0.7160.
137/937 [===>..........................] - ETA: 12:45 - loss: 0.1343 - accuracy: 0.9462For batch 137, loss is  0.0146.


206/937 [=====>........................] - ETA: 11:19 - loss: 0.0917 - accuracy: 0.9638For batch 206, loss is  0.0058.


275/937 [=======>......................] - ETA: 10:06 - loss: 0.0698 - accuracy: 0.9727For batch 275, loss is  0.0018.


413/937 [============>.................] - ETA: 7:53 - loss: 0.0470 - accuracy: 0.9818For batch 413, loss is  0.0005.


551/937 [================>.............] - ETA: 5:45 - loss: 0.0358 - accuracy: 0.9862For batch 551, loss is  0.0013.


...

549/937 [================>.............] - ETA: 5:40 - loss: 1.1784e-05 - accuracy: 1.0000For batch 549, loss is  0.0000.


683/937 [====================>.........] - ETA: 3:42 - loss: 1.1612e-05 - accuracy: 1.0000For batch 683, loss is  0.0000.


817/937 [=========================>....] - ETA: 1:45 - loss: 1.1349e-05 - accuracy: 1.0000For batch 817, loss is  0.0000.


936/937 [============================>.] - ETA: 0s - loss: 1.1214e-05 - accuracy: 1.0000For batch 936, loss is  0.0000.
 For batch 0, loss is  0.0344.
 For batch 1, loss is  0.0329.
 For batch 2, loss is  0.0228.
 For batch 3, loss is  0.0055.
 For batch 4, loss is  0.0267.
 For batch 5, loss is  0.0102.
 For batch 6, loss is  0.0218.
 For batch 7, loss is  0.0550.
 For batch 8, loss is  0.0141.
 For batch 9, loss is  0.0054.
 For batch 10, loss is  0.0066.
 For batch 11, loss is  0.0191.
 For batch 12, loss is  0.0069.
 For batch 13, loss is  0.0437.
 For batch 14, loss is  0.0105.
 For batch 15, loss is  0.0230.
 For batch 16, loss is  0.0073.
 For batch 17, loss is  0.0491.
 For batch 18, loss is  0.0034.
 For batch 19, loss is  0.0030.
 For batch 20, loss is  0.0293.
 For batch 21, loss is  0.0164.
 For batch 22, loss is  0.0270.
 For batch 23, loss is  0.0106.
 For batch 24, loss is  0.0145.
 For batch 25, loss is  0.0221.
 For batch 26, loss is  0.0119.
 For batch 27, loss is  0.

 For batch 61, loss is  0.0342.
 For batch 62, loss is  0.0344.
 For batch 63, loss is  0.0195.
 For batch 64, loss is  0.0023.
 For batch 65, loss is  0.0275.
 For batch 66, loss is  0.0184.
 For batch 67, loss is  0.0148.
 For batch 68, loss is  0.0238.
 For batch 69, loss is  0.0025.
 For batch 70, loss is  0.0562.
 For batch 71, loss is  0.0157.
 For batch 72, loss is  0.0193.
 For batch 73, loss is  0.0199.
 For batch 74, loss is  0.0149.
 For batch 75, loss is  0.0044.
 For batch 76, loss is  0.0397.
 For batch 77, loss is  0.0072.
 For batch 78, loss is  0.0323.
 For batch 79, loss is  0.0166.
 For batch 80, loss is  0.0140.
 For batch 81, loss is  0.0253.
 For batch 82, loss is  0.0103.
 For batch 83, loss is  0.0597.
 For batch 84, loss is  0.0216.
 For batch 85, loss is  0.0284.
 For batch 86, loss is  0.0134.
 For batch 87, loss is  0.0203.
 For batch 88, loss is  0.0084.
 For batch 89, loss is  0.0234.
 For batch 90, loss is  0.0142.
 For batch 91, loss is  0.0275.
 For bat

In [25]:
testGen.reset()
predIdxs = model.predict_generator(testGen,
                                        steps=(totalTest // batch_size) + 1)
predIdxs = np.argmax(predIdxs, axis=1)

print(classification_report(testGen.classes, predIdxs, target_names=testGen.class_indices.keys()))
print(confusion_matrix(testGen.classes, predIdxs))

              precision    recall  f1-score   support

    negative       1.00      1.00      1.00     14969
    positive       1.00      1.00      1.00     15031

    accuracy                           1.00     30000
   macro avg       1.00      1.00      1.00     30000
weighted avg       1.00      1.00      1.00     30000

[[14954    15]
 [    1 15030]]


In [26]:
model = tf.keras.applications.ResNet50(
    include_top=True, weights=None, input_tensor=None,
    input_shape=(160, 120, 3), pooling=None, classes=2)

model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['accuracy'])

In [27]:
starter = model.fit_generator(
    trainGen,
    steps_per_epoch=totalTrain // batch_size,
    validation_data=valGen,
    validation_steps=totalVal // batch_size,
    validation_freq=1,
    epochs=epochs,
callbacks=[LossAndErrorPrintingCallback()])

Train for 937 steps, validate for 117 steps
Epoch 1/10
For batch 0, loss is  0.8348.
 69/937 [=>............................] - ETA: 13:13 - loss: 0.9082 - accuracy: 0.7692For batch 69, loss is  0.0926.


207/937 [=====>........................] - ETA: 10:36 - loss: 0.3231 - accuracy: 0.9158For batch 207, loss is  0.0562.


345/937 [==========>...................] - ETA: 8:31 - loss: 0.1980 - accuracy: 0.9481For batch 345, loss is  0.0013.


483/937 [==============>...............] - ETA: 6:31 - loss: 0.1433 - accuracy: 0.9623For batch 483, loss is  0.0018.


621/937 [==================>...........] - ETA: 4:31 - loss: 0.1125 - accuracy: 0.9704For batch 621, loss is  0.0001.


...

552/937 [================>.............] - ETA: 5:28 - loss: 1.3807e-06 - accuracy: 1.0000For batch 552, loss is  0.0000.


686/937 [====================>.........] - ETA: 3:34 - loss: 1.3832e-06 - accuracy: 1.0000For batch 686, loss is  0.0000.


820/937 [=========================>....] - ETA: 1:39 - loss: 1.3496e-06 - accuracy: 1.0000For batch 820, loss is  0.0000.


936/937 [============================>.] - ETA: 0s - loss: 1.3934e-06 - accuracy: 1.0000For batch 936, loss is  0.0000.
 For batch 0, loss is  0.0196.
 For batch 1, loss is  0.0155.
 For batch 2, loss is  0.0017.
 For batch 3, loss is  0.0051.
 For batch 4, loss is  0.0002.
 For batch 5, loss is  0.0042.
 For batch 6, loss is  0.0001.
 For batch 7, loss is  0.0224.
 For batch 8, loss is  0.0004.
 For batch 9, loss is  0.0012.
 For batch 10, loss is  0.0057.
 For batch 11, loss is  0.0068.
 For batch 12, loss is  0.0001.
 For batch 13, loss is  0.0070.
 For batch 14, loss is  0.0097.
 For batch 15, loss is  0.0015.
 For batch 16, loss is  0.0041.
 For batch 17, loss is  0.0020.
 For batch 18, loss is  0.0018.
 For batch 19, loss is  0.0000.
 For batch 20, loss is  0.0001.
 For batch 21, loss is  0.0010.
 For batch 22, loss is  0.0105.
 For batch 23, loss is  0.0070.
 For batch 24, loss is  0.0054.
 For batch 25, loss is  0.0011.
 For batch 26, loss is  0.0002.
 For batch 27, loss is  0.

 For batch 72, loss is  0.0002.
 For batch 73, loss is  0.0001.
 For batch 74, loss is  0.0016.
 For batch 75, loss is  0.0000.
 For batch 76, loss is  0.0002.
 For batch 77, loss is  0.0011.
 For batch 78, loss is  0.0008.
 For batch 79, loss is  0.0000.
 For batch 80, loss is  0.0026.
 For batch 81, loss is  0.0015.
 For batch 82, loss is  0.0000.
 For batch 83, loss is  0.0003.
 For batch 84, loss is  0.0074.
 For batch 85, loss is  0.0006.
 For batch 86, loss is  0.0016.
 For batch 87, loss is  0.0000.
 For batch 88, loss is  0.0000.
 For batch 89, loss is  0.0015.
 For batch 90, loss is  0.0000.
 For batch 91, loss is  0.0018.
 For batch 92, loss is  0.0021.
 For batch 93, loss is  0.0008.
 For batch 94, loss is  0.0026.
 For batch 95, loss is  0.0171.
 For batch 96, loss is  0.0391.
 For batch 97, loss is  0.0065.
 For batch 98, loss is  0.0056.
 For batch 99, loss is  0.0003.
 For batch 100, loss is  0.0004.
 For batch 101, loss is  0.0000.
 For batch 102, loss is  0.0128.
 For 

In [28]:
from sklearn.metrics import classification_report, confusion_matrix
testGen.reset()
predIdxs = model.predict_generator(testGen,
                                        steps=(totalTest // batch_size) + 1)
predIdxs = np.argmax(predIdxs, axis=1)

print(classification_report(testGen.classes, predIdxs, target_names=testGen.class_indices.keys()))
print(confusion_matrix(testGen.classes, predIdxs))

              precision    recall  f1-score   support

    negative       1.00      0.97      0.98     14969
    positive       0.97      1.00      0.98     15031

    accuracy                           0.98     30000
   macro avg       0.98      0.98      0.98     30000
weighted avg       0.98      0.98      0.98     30000

[[14459   510]
 [   35 14996]]
